# Initialising the Lead Employer Agent

## If you wish to start at the beginning of the Doctor's in Training flow go [here](http://localhost:8889/notebooks/Part%201%20-%20Getting%20Started.ipynb).

In this notebook you will register the DID of the Lead Employer agent on the Sovrin StagingNet and write the credential definitions for the schema the Lead Employer is responsible for issuing.

There are:
* Occupational Health (Vaccination): Schema_id - [2yWoMSfeTtYBFpUJFcnpqL:2:Occupational Health:0.0.1](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/141273)
* Mandatory Training Credential: Schema_id - [2yWoMSfeTtYBFpUJFcnpqL:2:mandatoryTrainingCredential:0.0.1](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/141271)
* DBS Check: Schema id - [2yWoMSfeTtYBFpUJFcnpqL:2:DBS Criminal Record Check:0.0.1](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/141274)
* Lead Employer Assignments: Schema id - [2yWoMSfeTtYBFpUJFcnpqL:2:Lead Employer Assignment:0.0.1](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/141275)


## 1. Initialise a controller for Lead Employer Agent

In [4]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8062
ADMIN_URL = "http://lead-employer-agent:8061"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


## 2. Get current public DID

Before being able to write to any indy based ledger, your agent must have a public DID written on the ledger giving it the authority to write to it. As the cell below shows, this agent does not currently have a public DID. So any writes to the ledger will be rejected.

In [5]:
response = await agent_controller.wallet.get_public_did()
print(response)

{'result': {'did': 'U1YznPbdN3yuwPZadpNJf7', 'verkey': 'Fipnbu4eb5TE7g163BmGE7b8mpQeGVYvoMmdusjpYNyR', 'public': True}}


## 3. Generate a new DID

Before being able to write a DID to the ledger, you must create one using the wallet api. This api returns the identifier (the DID), and the verification key for that DID. A representation of it's public key. 

In [6]:
# generate new DID
response = await agent_controller.wallet.create_did()

did_object = response['result']
print("New DID", did_object)

New DID {'did': '8L5ggm8DnaRUA94aW4YRUG', 'verkey': '4zgeB1PFXJvNZKUATT9xVKa3RZ5tCKzJthVAfVKWRbWQ', 'public': False}


## 4. Write DID to Sovrin Stagingnet

Anoyone can write a DID to the Sovrin StagingNet, it is a permissionless ledger. 

Visit [Sovrin Selfserve Portal](https://selfserve.sovrin.org) for more information. We have provided an automated process to write DIDs to Stagingnet in the step below.

In [7]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())
print(r.status_code)

{'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'body': '{"statusCode": 200, "8L5ggm8DnaRUA94aW4YRUG": {"status": "Success", "statusCode": 200, "reason": "Successfully wrote NYM identified by 8L5ggm8DnaRUA94aW4YRUG to the ledger with role ENDORSER"}}'}
200


## 5. Accepting the Transaction Author Agreement (TAA)

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [8]:
response = await agent_controller.ledger.get_taa()
TAA_text = response['result']['taa_record']['text']
TAA_version = response['result']['taa_record']['version']
TAA_digest = response['result']['taa_record']['digest']


taa_record = {
            "version": TAA_version,
            "mechanism": "service_agreement",
            'text': TAA_text
}

TAA_to_sign = json.dumps(taa_record)

In [9]:
response = await agent_controller.ledger.accept_taa(TAA_to_sign)
## Will return {} if successful
print(response)

{}


## 6. Set public DID

Now you are able to assign the DID written to the ledger as public.

In [10]:
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

{'result': {'did': '8L5ggm8DnaRUA94aW4YRUG', 'verkey': '4zgeB1PFXJvNZKUATT9xVKa3RZ5tCKzJthVAfVKWRbWQ', 'public': True}}


## 7. Write Credential Definition for the Occupational Health to Ledger

The Health Education England must have a credential definition on the ledger to issue this credential.



In [11]:
occ_health_schema_id = "2yWoMSfeTtYBFpUJFcnpqL:2:Occupational Health:0.0.1"
response = await agent_controller.definitions.write_cred_def(occ_health_schema_id)

occ_health_cred_def_id = response["credential_definition_id"]
print(occ_health_cred_def_id)

%store occ_health_schema_id
%store occ_health_cred_def_id

8L5ggm8DnaRUA94aW4YRUG:3:CL:141273:default
Stored 'occ_health_schema_id' (str)
Stored 'occ_health_cred_def_id' (str)


## 8. Write Credential Definition for the Mandatory Training to Ledger

The Health Education England must have a credential definition on the ledger to issue this credential.


In [12]:
mandatory_training_schema_id = "2yWoMSfeTtYBFpUJFcnpqL:2:mandatoryTrainingCredential:0.0.1"
response = await agent_controller.definitions.write_cred_def(mandatory_training_schema_id)

mandatory_training_cred_def_id = response["credential_definition_id"]
print(mandatory_training_cred_def_id)

%store mandatory_training_schema_id
%store mandatory_training_cred_def_id

8L5ggm8DnaRUA94aW4YRUG:3:CL:141271:default
Stored 'mandatory_training_schema_id' (str)
Stored 'mandatory_training_cred_def_id' (str)


## 9. Write Credential Definition for the DBS Check schema to Ledger

The Health Education England must have a credential definition on the ledger to issue this credential.


In [13]:
dbs_schema_id = "2yWoMSfeTtYBFpUJFcnpqL:2:DBS Criminal Record Check:0.0.1"
response = await agent_controller.definitions.write_cred_def(dbs_schema_id)
dbs_cred_def_id = response["credential_definition_id"]
print(dbs_cred_def_id)

%store dbs_schema_id
%store dbs_cred_def_id

8L5ggm8DnaRUA94aW4YRUG:3:CL:141274:default
Stored 'dbs_schema_id' (str)
Stored 'dbs_cred_def_id' (str)


## 10. Write a Cred Def for the Lead Employer Assignment Schema

In [14]:
lead_emp_schema_id = "2yWoMSfeTtYBFpUJFcnpqL:2:Lead Employer Assignment:0.0.1"
response = await agent_controller.definitions.write_cred_def(lead_emp_schema_id)
lead_emp_cred_def_id = response["credential_definition_id"]
print(lead_emp_cred_def_id)

%store lead_emp_schema_id
%store lead_emp_cred_def_id

8L5ggm8DnaRUA94aW4YRUG:3:CL:141275:default
Stored 'lead_emp_schema_id' (str)
Stored 'lead_emp_cred_def_id' (str)


## 12. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [15]:
response = await agent_controller.terminate()
print(response)

None


# Proceed to [Part 5.2](http://localhost:8892/notebooks/Part%205.2%20-%20Onboard%20a%20Doctor%20in%20Training.ipynb) where you will be able to onboard Doctors in Training into your employ.

Typically when developing an SSI application, this configuration only needs to happen once. Then the agent would be ready to issue these credentials to all future doctor interactions.
